# Setup an instance with a default schema

In [ ]:
from lndb_setup import init, load
from lndb_setup import settings
from lndb_setup._clone import setup_local_test_postgres
from pathlib import Path

Assume we already setup a user account for "testuser1@lamin.ai" an chose the handle `testuser1` on [https://lamin.ai/settings/profile](https://lamin.ai/settings/profile).

In [ ]:
!lndb login testuser1

Spin up a local postgres for testing, we'll need it later.

In [ ]:
pgurl = setup_local_test_postgres()

## Initialize a LaminDB instance

### Init local SQLite with local storage

Running the following on the command line
```
lndb init --storage "mydata"
```
runs this:

In [ ]:
init(storage="mydata")

In [ ]:
settings.instance

In [ ]:
settings.user

In [ ]:
settings.instance.name

In [ ]:
assert settings.instance.cloud_storage == False
assert settings.instance.owner == settings.user.handle
assert settings.instance.name == "mydata"
assert settings.instance.storage_root.as_posix() == Path("mydata").absolute().as_posix()
assert settings.instance.cache_dir is None

### Init postgres with local storage

The connection string for postgres looks like this (we created it above):

In [ ]:
pgurl

Let us call init:

In [ ]:
init(storage="mydatapg", url=pgurl)

Clean up:

In [ ]:
!docker stop pgtest && docker rm pgtest

### Custom instance name

You can also provide a name for the instance
```
lndb init --storage "mydata_storage" --name "mydata_instance_name"
```
which is equivalent to this:

In [ ]:
init(storage="mydata_storage", name="mydata_instance_name")

In [ ]:
assert settings.instance.name == "mydata_instance_name"

### Configure with cloud storage on AWS

Now on AWS S3 (`lndb init --storage "s3://lndb-setup-ci"`). You need to have setup AWS via `awscli configure`.

In [ ]:
init(storage="s3://lndb-setup-ci")

In [ ]:
settings.instance

In [ ]:
settings.instance.name

In [ ]:
settings.instance.storage_root

In [ ]:
settings.instance.storage_region

In [ ]:
settings.instance.cache_dir

In [ ]:
settings.instance.db

In [ ]:
settings.instance._sqlite_file

In [ ]:
settings.instance._sqlite_file_local

In [ ]:
assert settings.instance.cloud_storage == True
assert str(settings.instance.storage_root) == "s3://lndb-setup-ci"
assert settings.instance.storage_region == "us-east-1"
assert str(settings.instance._sqlite_file) == "s3://lndb-setup-ci/lndb-setup-ci.lndb"

Do the same for an S3 bucket in Europe.

In [ ]:
init(storage="s3://lndb-setup-ci-eu-central-1")

In [ ]:
assert settings.instance.storage_region == "eu-central-1"

Clean up.

In [ ]:
settings.instance._sqlite_file.unlink()
settings.instance._sqlite_file_local.unlink()

### Configure with cloud storage on GCP

You need to authenticate for Google Clod.

* Either, set the environment variable `export GOOGLE_APPLICATION_CREDENTIALS=<HOME-DIR>/.lndb/<GOOGLE CLOUD PROJECT>.json`.
* Alternatively, if you set up the `gcloud` CLI, log in with `gcloud auth application-default login`.


In [ ]:
init(storage="gs://lndb-setup-ci-us")

Clean up.

In [ ]:
settings.instance._sqlite_file.unlink()
settings.instance._sqlite_file_local.unlink()

## Load an existing instance

Running the following on the command line
```
lndb load mydata
```
runs this:

In [ ]:
load("mydata")

In [ ]:
import os

assert settings.instance.cloud_storage == False
assert settings.instance.name == "mydata"
assert settings.instance.storage_root.as_posix() == f"{os.getcwd()}/mydata"
assert settings.instance.cache_dir is None
assert settings.instance.db == f"sqlite:///{os.getcwd()}/mydata/mydata.lndb"

Clean up the test case.

In [ ]:
settings.instance._sqlite_file.unlink()

Running the following on the command line
```
lndb load mydata_instance_name
```
runs this:

In [ ]:
load("mydata_instance_name")

In [ ]:
import os

assert settings.instance.cloud_storage == False
assert settings.instance.name == "mydata_instance_name"
assert settings.instance.storage_root.as_posix() == f"{os.getcwd()}/mydata_storage"
assert settings.instance.cache_dir is None
assert (
    settings.instance.db
    == f"sqlite:///{os.getcwd()}/mydata_storage/mydata_instance_name.lndb"
)

Clean up the test case.

In [ ]:
settings.instance._sqlite_file.unlink()

Running the following on the command line
```
lndb load lndb-setup-ci
```
runs this:

In [ ]:
load("lndb-setup-ci")

In [ ]:
assert settings.instance.cloud_storage == True
assert str(settings.instance.storage_root) == "s3://lndb-setup-ci"
assert str(settings.instance._sqlite_file) == "s3://lndb-setup-ci/lndb-setup-ci.lndb"

Clean up the test case.

In [ ]:
settings.instance._sqlite_file.unlink()
settings.instance._sqlite_file_local.unlink()